In [1]:
import csv
import os
import traceback
import numpy as np
import tqdm
import pandas as pd
import multiprocessing as mp
import gc
from datetime import timedelta, date
import uuid

# import rsa_data_summary as rd
# import rsa_data_wim as wim
# import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)

In [2]:
# individual tables for type 30 and 60 test
# files = tools.getfiles(r'C:\FTP\Syntell\SMEC RSA Files_GP PRM Sites_Dec21toFeb22')
file = r"C:\PQ410\2 - Data In\TIS Data\GDRT\GP_RSA 2014\0038.RSA"
t10_file = r"C:\FTP\Syntell\SMEC RSV Files_GP PRM Sites_Dec21toFeb22_individuals\0006-20211231.RSV"
# file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"
type60_file = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\type21_60_70.RSA"

In [3]:
# electronic_count_data_type_21_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_21 limit 1",config.ENGINE).columns)
# electronic_count_data_type_30_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_30 limit 1",config.ENGINE).columns)
electronic_count_data_type_60_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_60 limit 1",config.ENGINE).columns)
# electronic_count_data_type_70_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_data_type_70 limit 1",config.ENGINE).columns)
# header_columns = list(pd.read_sql_query("SELECT * from trafc.electronic_count_header limit 1",config.ENGINE).columns)

In [3]:
df = tools.to_df(file)

In [24]:
data_df = pd.DataFrame(df.loc[
            (~df[0].isin(["H0", "H9", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
            & (
                (df[0].isin(["21", "22", "70", "30", "31", "13", "60"]))
                & (df[1].isin(["0", "1", "2", "3", "4"]))
                & ((df.loc[df[3].str.len() > 3]).all()[0])
            )
        ]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [35]:
head_df = pd.DataFrame(df.loc[
    (df[0].isin(["H0", "S0", "I0", "S1", "D0", "D1", "D3", "L0", "L1"]))
    | (
        (df[0].isin(["21", "70", "30", "13", "60"]))
        & (df.loc[df[2].isin(["21", "70", "30", "13", "60"])][3].astype(int) < 24)
    )
    | (
        (df[0].isin(["10"]))
        & (df[1].isin(["1", "8", "5", "9", "01", "08", "05", "09"]))
    )
]).dropna(axis=1, how="all").reset_index(drop=True).copy()

In [36]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,10
0,H0,200,002,RSA,Data,Format,Version,2.00,None,None,None
1,S0,0038,0038,Lion,Park,None,None,None,None,None,None
2,S1,R512,1.2,00,0,Between,P158/2,(R28/N14),&,P39/1,(R114)
3,I0,RT8010,Fam,110909MM,None,None,None,None,None,None,None
4,D0,M,L,None,None,None,None,None,None,None,None
5,D1,140503,074727,140511,090000,140503,074700,140503,074726,140511,090000
6,D3,20140503,074727,20140511,090000,20140503,074700,20140503,074726,20140511,090000
7,L0,04,04,None,None,None,None,None,None,None,None
8,L1,01,0,1,00,0,1,1,1,0,0
9,L1,02,0,2,00,0,1,1,1,0,0


In [7]:
st_nd = df.loc[df[0].isin(["D1","D3"]), 0:4].reset_index(drop=True).copy()

# adds century to year if it is not there
st_nd[1] = st_nd[1].apply(lambda x: str(date.today())[:2] + x if len(x)==6 else x)
st_nd[3] = st_nd[3].apply(lambda x: str(date.today())[:2] + x if len(x)==6 else x)

st_nd[1] = st_nd[1].str.pad(width=8, side='right', fillchar="0")
st_nd[3] = st_nd[3].str.pad(width=8, side='right', fillchar="0")

# index 2 and 4 are time, this makes the time uniform length
st_nd[2] = st_nd[2].str.pad(width=7,side='right',fillchar="0")
st_nd[4] = st_nd[4].str.pad(width=7,side='right',fillchar="0")

# this filters for time = 24H00m and makes it zero hour
st_nd[2].loc[st_nd[2].str[:2] == '24'] = ('0').zfill(7)
st_nd[4].loc[st_nd[4].str[:2] == '24'] = ('0').zfill(7)

# adds a day if the hour is zero hour and changes string to dtetime.date
try:
    st_nd[1] = st_nd[1].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
    if x[2] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
    st_nd[3] = st_nd[3].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
    if x[4] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
# except ValueError:
#     st_nd[1] = st_nd[1].apply(lambda x: pd.to_datetime(x[:8], format="%Y%m%d").date() + timedelta(days=1)
#     if x[2] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x[:8], format="%Y%m%d").date())
#     st_nd[3] = st_nd[3].apply(lambda x: pd.to_datetime(x[:8], format="%Y%m%d").date() + timedelta(days=1)
#     if x[4] in ['0'.zfill(7),'24'.zfill(7)] else pd.to_datetime(x[:8], format="%Y%m%d").date())
except:
    pass

# changes time string into datetime.time
try:
    st_nd[2] = st_nd[2].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
    st_nd[4] = st_nd[4].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
except ValueError:
    st_nd[2] = st_nd[2].apply(lambda x: pd.to_datetime(x[:7], format="%H%M%S%f").time())
    st_nd[4] = st_nd[4].apply(lambda x: pd.to_datetime(x[:7], format="%H%M%S%f").time())
except:
    pass

# creates start_datetime and end_datetime
try:
    st_nd["start_datetime"] = pd.to_datetime((st_nd[1].astype(str)+st_nd[2].astype(str)), 
        format='%Y-%m-%d%H:%M:%S')
    st_nd["end_datetime"] = pd.to_datetime((st_nd[3].astype(str)+st_nd[4].astype(str)), 
        format='%Y-%m-%d%H:%M:%S')
except Exception:
    pass


st_nd = st_nd.iloc[:,1:].drop_duplicates()

headers = pd.DataFrame()
try:
    headers['start_datetime'] = st_nd.groupby(st_nd['start_datetime'].dt.year).min()['start_datetime']
    headers['end_datetime'] = st_nd.groupby(st_nd['end_datetime'].dt.year).max()['end_datetime']
except:
    pass

headers['site_id'] = df.loc[df[0] == "S0",1].drop_duplicates().reset_index(drop=True)[0]
headers['site_id'] = headers['site_id'].astype(str)

headers["header_id"] = ""
headers["header_id"] = headers["header_id"].apply(
            lambda x: str(uuid.uuid4()))

headers['year'] = headers['start_datetime'].dt.year

headers["number_of_lanes"] = int(df.loc[df[0] == "L0", 2].drop_duplicates().reset_index(drop=True)[0])

station_name = df.loc[df[0].isin(["S0"]), 3:].reset_index(drop=True).drop_duplicates().dropna(axis=1)
headers["station_name"] = station_name[station_name.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

t21 = df.loc[df[0]=="21"].dropna(axis=1).drop_duplicates().reset_index().copy()
t21 = t21.iloc[:,(t21.shape[1])-9:].astype(int)
t21.columns = range(t21.columns.size)
t21 = t21.rename(columns={
    0:'speedbin1',
    1:'speedbin2',
    2:'speedbin3',
    3:'speedbin4',
    4:'speedbin5',
    5:'speedbin6',
    6:'speedbin7',
    7:'speedbin8',
    8:'speedbin9'}
    ,inplace = True)

In [12]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,H0,200,002,RSA,Data,Format,Version,2.00,None,None,None
1,S0,0038,0038,Lion,Park,None,None,None,None,None,None
2,S1,R512,1.2,00,0,Between,P158/2,(R28/N14),&,P39/1,(R114)
3,I0,RT8010,Fam,110909MM,None,None,None,None,None,None,None
4,D0,M,L,None,None,None,None,None,None,None,None


In [ ]:
def process_data_datetimes(df: pd.DataFrame) -> pd.DataFrame:
    date_length = len(df[2].at[0])
    duration_min = int(df[4].at[0])

    df[3] = df[3].astype(str)

    df[3] = df[3].str.pad(width=7,side='right',fillchar="0")
    df[3].loc[df[3].str[:2] == '24'] = ('0').zfill(7)

    if date_length == 6:
        decade = int(df[2].at[0][:2])
        if decade < 50:
            century = str(date.today())[:2]
        else:
            century = int(str(date.today())[:2])-1

        df[2] = str(century) + df[2]
        df[2] = df[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] in ['0'.zfill(7),'24'.ljust(7,'0')] else pd.to_datetime(x, format="%Y%m%d").date())
    elif date_length == 8:
        df[2] = df[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] in ['0'.zfill(7),'24'.ljust(7,'0')] else pd.to_datetime(x, format="%Y%m%d").date())
    else:
        raise Exception("DATA Date length abnormal")

    df[3] = df[3].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())

    try:
        df['end_datetime'] = pd.to_datetime(df[2].astype(str)+df[3].astype(str), 
            format='%Y-%m-%d%H:%M:%S.%f')
    except ValueError:
        df['end_datetime'] = pd.to_datetime((df[2].astype(str)+df[3].astype(str))[:18], 
            format='%Y-%m-%d%H:%M:%S')

    try:
        df['start_datetime'] = pd.to_datetime(df[2].astype(str)+df[3].astype(str), 
            format='%Y-%m-%d%H:%M:%S.%f') - timedelta(minutes=duration_min)
    except ValueError:
        df['start_datetime'] = pd.to_datetime((df[2].astype(str)+df[3].astype(str))[:18], 
            format='%Y-%m-%d%H:%M:%S') - timedelta(minutes=duration_min)

    df.rename(columns={
        2:"end_date",
        3:"end_time"
    },inplace = True)

    return df

def get_direction(df):
    try:
        dir_1 = self.lanes["lane_number"].astype(int).min()
        dir_2 = self.lanes["lane_number"].astype(int).max()
    except (TypeError,ValueError):
        dir_1 = 0
        dir_2 = 4

    df['direction'] = df[5]
    df['compass_heading'] = df[5]
    df['direction_code'] = df[5]
    df['direction'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_1]))] = 'P'
    df['direction'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_2]))] = 'N'
    df['compass_heading'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_1]))] = str(dir_1)
    df['compass_heading'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_2]))] = str(dir_2)
    df['direction_code'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_1]))] = str(dir_1)
    df['direction_code'].loc[df[5].isin(list(self.lanes['lane_number'].astype(str).loc[self.lanes['direction_code']==dir_2]))] = str(dir_2)
    
    return df



In [23]:
data = data_df.loc[(data_df[0] == "30")].dropna(
            axis=1, how="all"
        ).reset_index(drop=True).copy()
header = head_df.loc[(head_df[0] == "30")].dropna(
            axis=1, how="all"
        ).reset_index(drop=True).copy()


KeyError: "['start_datetime', 'end_date', 'end_time', 'direction', 'compass_heading'] not in index"

In [38]:

if header.shape[1] > 3:
    classification_scheme = header.iloc[0,3]
    number_of_data_records = header.iloc[0,4]
else:
    classification_scheme = header.iloc[0,2]
    number_of_data_records = header.iloc[0,3]

# vc_df = select_classification_scheme(classification_scheme)

if data[1].isin(["0", "2", 0, 2]).any():
    ddf = data.iloc[:, 1:].dropna(axis=1, how="all").reset_index(drop=True)

    duration_min = int(ddf[4][0])
    # max_lanes = self.lanes['lane_number'].astype(int).max()

    ddf[ddf.select_dtypes(include=['object']).columns] = ddf[
        ddf.select_dtypes(include=['object']).columns].apply(
        pd.to_numeric, axis=1, errors='ignore')
        
    ddf['vehicle_classification_scheme'] = int(classification_scheme)

    ddf.columns = ddf.columns.astype(str)

    df3 = pd.DataFrame(columns=[
        'edit_code', 
        'start_datetime', 
        'end_date', 
        'end_time', 
        'duration_of_summary', 
        'lane_number', 
        'number_of_vehicles', 
        'class_number', 
        'direction', 
        'compass_heading'])
    for lane_no in range(1, max_lanes+1):
        for i in range(6,int(number_of_data_records)+6):
            print(lane_no)
            join_to_df3 = ddf.loc[ddf['5'].astype(int) == lane_no, ['1', 'start_datetime','end_date', 'end_time', '4', '5',str(i), 'direction', 'compass_heading']]
            print(ddf.loc[ddf['5'] == lane_no, ['1', 'start_datetime','end_date', 'end_time', '4', '5',str(i), 'direction', 'compass_heading']])
            join_to_df3['class_number'] = i-5
            print(join_to_df3)
            join_to_df3.rename(columns={
                '1':"edit_code",
                '2':"end_date",
                '3':"end_time",
                '4':"duration_of_summary",
                '5':'lane_number', 
                str(i): 'number_of_vehicles'
                }, inplace=True)
            df3 = pd.concat([df3,join_to_df3],keys=['start_datetime','lane_number','number_of_vehicles','class_number'],ignore_index=True, axis=0)
            # df3 = pd.concat([df3,join_to_df3] ,ignore_index=True, axis=0)
    # df3 = df3.apply(pd.to_numeric, axis=1, errors="ignore")
    # df3['classification_scheme'] = int(classification_scheme)
    # df3['site_id'] = self.site_id
    # df3['year'] = int(df3['start_datetime'].at[0].year)


1
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 6, direction, compass_heading]
Index: []
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 6, direction, compass_heading, class_number]
Index: []
1
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 7, direction, compass_heading]
Index: []
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 7, direction, compass_heading, class_number]
Index: []
1
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 8, direction, compass_heading]
Index: []
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 8, direction, compass_heading, class_number]
Index: []
1
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 9, direction, compass_heading]
Index: []
Empty DataFrame
Columns: [1, start_datetime, end_date, end_time, 4, 5, 9, direction, compass_heading, class_number]
Index: []
1
Empty DataFrame
Columns: [1, start_dat

In [49]:
ddf.loc[ddf['5'].astype(int) == 3]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,start_datetime,end_date,end_time,direction,compass_heading,vehicle_classification_scheme
2,0,220301,1300,15,03,0019,0000,0303,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,0000,start_datetime,end_date,end_time,direction,compass_heading,8


In [54]:
if (head_df[head_df[0] == "21"].empty) or (head_df[head_df[0] == "70"].empty) or (head_df[head_df[0] == "30"].empty) or (head_df[head_df[0] == "30"].empty):
    print('dont do something')
else:
    print('do something')

do something


In [13]:
header.loc[header[0]=="L1"].dropna(axis=1).drop_duplicates()

,0,1,2,3,4,5,6,7,8,9,10
7,L1,01,0,1,00,0,1,1,1,1,1
8,L1,02,0,2,00,0,1,1,1,0,0
9,L1,03,4,2,00,0,1,1,1,0,0
10,L1,04,4,1,00,0,1,1,1,1,1


In [5]:
def select_classification_scheme(classification_scheme):
    if int(classification_scheme) == 8:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_8, config.ENGINE)
    elif int(classification_scheme) == 1:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_1, config.ENGINE)
    elif int(classification_scheme) == 5:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_5, config.ENGINE)
    elif int(classification_scheme) == 9:
        vc_df = pd.read_sql_query(q.SELECT_CLASSIFICAITON_SCHEME_9, config.ENGINE)
    else:
        vc_df = None
    return vc_df
    

In [6]:
data = df.loc[(df[0] == "60") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
    axis=1, how="all"
).reset_index(drop=True)
dfh = pd.DataFrame(
    df.loc[
        (df[0].isin(["60"]))
        & (~df[1].isin(["0", "1", "2", "3", "4"]))
    ]
).dropna(axis=1, how="all").drop_duplicates().reset_index(drop=True).copy()
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
).reset_index(drop=True).copy()

In [8]:
lanes_df = header.loc[
    (header[0].isin(["L0", "L1", "L2", "L3","L4" ,"L5","L6","L7","L8","L9","L10","L11","L12"]))
].dropna(axis=1).reset_index(drop=True).drop_duplicates()

In [16]:
def process_dates_and_times(ddf: pd.DataFrame, date_length: int, time_length: int, duration_min: int):
    ddf[3] = ddf[3].astype(str)
    ddf[3].loc[ddf[3].str[:2] == '24'] = ('0').zfill(7)

    if date_length == 6:
        ddf[2] = str(date.today())[:2] + ddf[2]
        ddf[2] = ddf[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] in ['24'.zfill(7),'0'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
    elif date_length == 8:
        ddf[2] = ddf[2].apply(lambda x: pd.to_datetime(x, format="%Y%m%d").date() + timedelta(days=1)
            if x[3] == ['24'.zfill(7),'0'.zfill(7)] else pd.to_datetime(x, format="%Y%m%d").date())
    else:
        raise Exception("DATA Date length abnormal")

    if time_length < 7:
        ddf[3] = ddf[3].str.pad(width=7,side='right',fillchar="0")
        ddf[3] = ddf[3].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())
    else:
        ddf[3] = ddf[3].apply(lambda x: pd.to_datetime(x, format="%H%M%S%f").time())

    if (ddf[2].astype(str)+ddf[3].astype(str)).map(len).isin([20]).all():
        ddf['end_datetime'] = pd.to_datetime(ddf[2].astype(str)+ddf[3].astype(str), 
            format='%Y-%m-%d%H:%M:%S.%f')
    else:
        ddf['end_datetime'] = pd.to_datetime((ddf[2].astype(str)+ddf[3].astype(str))[:18], 
            format='%Y-%m-%d%H:%M:%S')

    if (ddf[2].astype(str)+ddf[3].astype(str)).map(len).isin([20]).all():
        ddf['start_datetime'] = pd.to_datetime(ddf[2].astype(str)+ddf[3].astype(str), 
            format='%Y-%m-%d%H:%M:%S.%f') - timedelta(minutes=duration_min)
    else:
        ddf['start_datetime'] = pd.to_datetime((ddf[2].astype(str)+ddf[3].astype(str))[:18], 
            format='%Y-%m-%d%H:%M:%S') - timedelta(minutes=duration_min)
    return ddf

In [17]:
    def electronic_count_data_type_60(df: pd.DataFrame) -> pd.DataFrame:    #  FOR TYPE 60
        data = df.loc[(df[0] == "60") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
            axis=1, how="all"
        ).reset_index(drop=True)
        dfh = pd.DataFrame(
            df.loc[
                (df[0].isin(["60"]))
                & (~df[1].isin(["0", "1", "2", "3", "4"]))
            ]
        ).dropna(axis=1, how="all").drop_duplicates().reset_index(drop=True).copy()
        dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
            axis=1, how="all"
        ).reset_index(drop=True).copy()

        if data.empty:
            pass
        else:
            dir_1 = dfh2.iloc[1:,2].astype(int).min()
            dir_2 = dfh2.iloc[1:,2].astype(int).max()

            number_of_data_records = dfh.iloc[0,3]

            if data[1].isin(["0", "1", "2", "3", "4"]).any():
                ddf = data.iloc[:, 1:].reset_index(drop=True)
                ddf = pd.DataFrame(ddf).dropna(axis=1, how="all").reset_index(drop=True)

                site_id = dfh2.iloc[1,1]

                time_length = len(ddf[2][0])
                date_length = len(ddf[2][0])
                duration_min = int(ddf[4][0])
                max_lanes = ddf[5].astype(int).max()

                dfh2 = dfh2.loc[dfh[0]=="L1"]
                direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(str)
                direction = direction.drop_duplicates()

                ddf = process_dates_and_times(ddf, date_length, time_length, duration_min)

                ddf['direction'] = ddf[5]
                ddf['compas_heading'] = ddf[5]
                ddf['direction'].loc[ddf[5].isin(list(direction[1].astype(str).loc[direction[2]==direction[2].min()]))] = 'P'
                ddf['direction'].loc[ddf[5].isin(list(direction[1].astype(str).loc[direction[2]==direction[2].max()]))] = 'N'
                ddf['compass_heading'].loc[ddf[5].isin(list(direction[1].astype(str).loc[direction[2]==direction[2].min()]))] = str(direction[2].min())
                ddf['compass_heading'].loc[ddf[5].isin(list(direction[1].astype(str).loc[direction[2]==direction[2].max()]))] = str(direction[2].max())

                ddf[ddf.select_dtypes(include=['object']).columns] = ddf[ddf.select_dtypes(include=['object']).columns].apply(pd.to_numeric, errors='ignore')

                ddf['start_datetime'] = pd.to_datetime(ddf[2].astype(str)+ddf[3].astype(str), 
                    format='%Y-%m-%d%H:%M:%S') - timedelta(minutes=duration_min)

                ddf['end_datetime'] = pd.to_datetime(ddf[2].astype(str)+ddf[3].astype(str), 
                    format='%Y-%m-%d%H:%M:%S')
            
                ddf.columns = ddf.columns.astype(str)

                dfh3 = dfh2.loc[dfh2[0]!="S0"].copy()
                df3 = pd.DataFrame(columns=[
                'edit_code', 
                'start_datetime', 
                'end_date', 
                'end_time', 
                'duration_of_summary', 
                'lane_number', 
                'bin_number', 
                'number_of_vehicles', 
                'bin_boundary_length_cm', 
                'direction', 
                'compass_heading'])

                for i in range(6,int(number_of_data_records)+6):
                    for lane_no in range(1, max_lanes+1):
                        join_to_df3 = ddf.loc[ddf['5'] == lane_no, [
                            '1', # edit_code
                            'start_datetime',
                            '2', # end_date
                            '3', # end_time
                            '4', # duration_of_summary
                            '5', # lane_number
                            str(i), # "number_of_vehicles"
                            'direction', 
                            'compass_heading'
                            ]]
                        join_to_df3['bin_number'] = str(i-5)
                        join_to_df3['bin_boundary_length_cm'] = int(dfh[i-2][0])
                        join_to_df3.rename(columns={
                            '1':"edit_code",
                            '2':"end_date",
                            '3':"end_time",
                            '4':"duration_of_summary",
                            '5':"lane_number",
                            str(i): "number_of_vehicles"
                            }, inplace=True)
                        df3 = pd.concat([df3,join_to_df3], axis=0, ignore_index=True)
                df3['site_id'] = site_id
                df3['year'] = int(df3['start_datetime'][0].year)

            else:
                pass
        return df3

In [33]:
ddf['6'].head()

0    20
1    40
2    52
3    58
4    80
Name: 6, dtype: int64

In [70]:
dfh3 = dfh2.loc[dfh2[0]!="S0"].copy()
df3 = pd.DataFrame(columns=[
'edit_code', 
'start_datetime', 
'end_date', 
'end_time', 
'duration_of_summary', 
'lane_number', 
'bin_number', 
'number_of_vehicles', 
'bin_boundary_length_cm', 
'direction', 
'compass_heading'])

for i in range(6,int(number_of_data_records)+6):
    for lane_no in range(1, max_lanes+1):
        join_to_df3 = ddf.loc[ddf['5'] == lane_no, [
            '1', # edit_code
            'start_datetime',
            '2', # end_date
            '3', # end_time
            '4', # duration_of_summary
            '5', # lane_number
            str(i), # "number_of_vehicles"
            'direction', 
            'compass_heading'
            ]]
        join_to_df3['bin_number'] = str(i-5)
        join_to_df3['bin_boundary_length_cm'] = int(dfh[i-2][0])
        join_to_df3.rename(columns={
            '1':"edit_code",
            '2':"end_date",
            '3':"end_time",
            '4':"duration_of_summary",
            '5':"lane_number",
            str(i): "number_of_vehicles"
            }, inplace=True)
        df3 = pd.concat([df3,join_to_df3], axis=0, ignore_index=True)


KeyError: "['compass_heading'] not in index"

In [69]:
df3.head(100)

,edit_code,start_datetime,end_date,end_time,duration_of_summary,lane_number,bin_number,number_of_vehicles,bin_boundary_length_cm,direction,compass_heading,compas_heading
0,0,2012-09-10 12:15:00,2012-09-10,12:30:00,15,1,1,20,400,P,NaN,0.0
1,0,2012-09-10 12:30:00,2012-09-10,12:45:00,15,1,1,19,400,P,NaN,0.0
2,0,2012-09-10 12:45:00,2012-09-10,13:00:00,15,1,1,27,400,P,NaN,0.0
3,0,2012-09-10 13:00:00,2012-09-10,13:15:00,15,1,1,14,400,P,NaN,0.0
4,0,2012-09-10 13:15:00,2012-09-10,13:30:00,15,1,1,14,400,P,NaN,0.0
5,0,2012-09-10 13:30:00,2012-09-10,13:45:00,15,1,1,19,400,P,NaN,0.0
6,0,2012-09-10 13:45:00,2012-09-10,14:00:00,15,1,1,20,400,P,NaN,0.0
7,0,2012-09-10 14:00:00,2012-09-10,14:15:00,15,1,1,20,400,P,NaN,0.0
8,0,2012-09-10 14:15:00,2012-09-10,14:30:00,15,1,1,15,400,P,NaN,0.0
9,0,2012-09-10 14:30:00,2012-09-10,14:45:00,15,1,1,17,400,P,NaN,0.0
